In [139]:
import os
import pandas as pd 
SUM_COL = ["clip 이름", "가공자", "검수일자", "검수결과", "반려사유", "id 개수", "객체 개수", "박스 개수", "오류 id 개수", "오류 박스 총계", "오류 id 목록"]
import shutil
import glob
super_base = r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill\작업완료클립"
csvs = [i for i in os.listdir( super_base ) if ((".csv" in i) and ("_sum" in i) )]


In [140]:
sum_log_df = pd.DataFrame(columns=SUM_COL)
for df_pth in csvs:
    df = pd.read_csv(f"{super_base}/{df_pth}")
    sum_log_df = pd.concat([sum_log_df, df], ignore_index=True)
    
sum_log_df.to_csv(r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill/concat_aivill.csv", index=False, encoding="utf-8-sig")


In [141]:
# os.listdir(r"Z:/Aivill_3F/Aivill_3F_유세록/00_할당")




# "Z:/Aivill_3F/Aivill_3F_유세록/00_할당/mmdd/S_Clip_00000_00/".split("/")




In [142]:
host = r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill/concat_aivill.csv"
scene = r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill/clip-name-convert.csv"
host_df = pd.read_csv(host, encoding="utf-8")
scene_df = pd.read_csv(scene, encoding="utf-8")


def get_scene(clip):
    ## 클립명에서 목표 객체 (동적or주행환경) 추출
    ## 0: 동적, 1: 주행환경
    cat_code = int(clip[-2:])
    if cat_code < 10:
        return 0
    else:
        return 1

In [143]:
host_df

,clip 이름,가공자,검수일자,검수결과,반려사유,id 개수,객체 개수,박스 개수,오류 id 개수,오류 박스 총계,오류 id 목록
0,extract_2022-08-02-18-18-53_Clip_00004,김태헌,2022-08-24,반려,메타정보 불일치,10,379,379,1,1,10
1,extract_2022-08-02-18-18-53_Clip_00011,조준혁,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",23,585,585,6,64,"4, 9, 10, 11, 16, 18"
2,extract_2022-08-02-18-18-53_Clip_00035,남준우,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",3,215,215,3,216,"1, 2, 3"
3,extract_2022-08-02-18-18-53_Clip_00056,남준우,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",6,333,333,6,342,"1, 2, 3, 4, 5, 6"
4,extract_2022-08-02-18-18-53_Clip_00001,김효중,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",18,941,941,5,209,"3, 5, 6, 7, 8"
...,...,...,...,...,...,...,...,...,...,...,...
182,extract_2022-08-03-14-48-26_Clip_00041,강려명,2022-08-24,통과,통과,1,100,100,0,0,없음
183,extract_2022-08-03-14-48-26_Clip_00043,강려명,2022-08-24,통과,통과,1,100,100,0,0,없음
184,extract_2022-08-03-15-44-05_Clip_00120,이정모,2022-08-24,통과,통과,2,200,200,0,0,없음
185,extract_2022-08-03-15-44-05_Clip_00140,강려명,2022-08-24,통과,통과,3,300,300,0,0,없음


In [144]:
scene_df["clip 이름"] = scene_df["extract"] + "_" + scene_df["clip"]
# scene_df["scene"] = get_scene( scene_df["folder"] )
scene_df["scene"] = 0
for i in range(len(scene_df)):
    clip = scene_df.loc[i]["folder"]
    tmp = get_scene( clip )
    if tmp == 0: tmp = "동적"
    else: tmp = "주행환경"
    scene_df.iloc[i, 5] = tmp
    
scene_df = scene_df.iloc[:,-2:]
scene_df

,clip 이름,scene
0,extract_2022-08-01-10-53-48_Clip_00000,주행환경
1,extract_2022-08-01-10-53-48_Clip_00001,주행환경
2,extract_2022-08-01-10-53-48_Clip_00002,주행환경
3,extract_2022-08-01-10-53-48_Clip_00003,주행환경
4,extract_2022-08-01-10-53-48_Clip_00004,주행환경
...,...,...
849,extract_2022-08-03-15-44-05_Clip_00150,동적
850,extract_2022-08-03-15-44-05_Clip_00151,동적
851,extract_2022-08-03-15-44-05_Clip_00152,동적
852,extract_2022-08-03-15-44-05_Clip_00153,동적


In [145]:
out_df = pd.merge(host_df, scene_df)
out_df["scene"].value_counts()

out_df

,clip 이름,가공자,검수일자,검수결과,반려사유,id 개수,객체 개수,박스 개수,오류 id 개수,오류 박스 총계,오류 id 목록,scene
0,extract_2022-08-02-18-18-53_Clip_00004,김태헌,2022-08-24,반려,메타정보 불일치,10,379,379,1,1,10,동적
1,extract_2022-08-02-18-18-53_Clip_00011,조준혁,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",23,585,585,6,64,"4, 9, 10, 11, 16, 18",동적
2,extract_2022-08-02-18-18-53_Clip_00035,남준우,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",3,215,215,3,216,"1, 2, 3",동적
3,extract_2022-08-02-18-18-53_Clip_00056,남준우,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",6,333,333,6,342,"1, 2, 3, 4, 5, 6",동적
4,extract_2022-08-02-18-18-53_Clip_00001,김효중,2022-08-24,반려,"비정형/정형 오류, 메타정보 불일치",18,941,941,5,209,"3, 5, 6, 7, 8",동적
...,...,...,...,...,...,...,...,...,...,...,...,...
182,extract_2022-08-03-15-44-05_Clip_00137,박세령,2022-08-24,통과,통과,3,300,300,0,0,없음,동적
183,extract_2022-08-03-14-48-26_Clip_00043,강려명,2022-08-24,통과,통과,1,100,100,0,0,없음,동적
184,extract_2022-08-03-15-44-05_Clip_00120,이정모,2022-08-24,통과,통과,2,200,200,0,0,없음,동적
185,extract_2022-08-03-15-44-05_Clip_00140,강려명,2022-08-24,통과,통과,3,300,300,0,0,없음,동적


In [146]:


# pass_df = out_df.loc[out_df['검수결과'] == "통과"]
# rej_df = out_df.loc[out_df['검수결과'] == "반려"]

# d_df = out_df.loc[out_df['scene'] == "동적"]
# s_df = out_df.loc[out_df['scene'] == "주행환경"]

# print("검수통과 중 동적/주행환경")
# print(pass_df["scene"].value_counts())
# print("\n")

# print("검수반려 중 동적/주행환경")
# print(rej_df["scene"].value_counts())
# print("\n")

# print("동적 중 통과/반려")
# print(d_df["검수결과"].value_counts())
# print("\n")

# print("주행환경 중 통과/반려")
# print(s_df["검수결과"].value_counts())
# print("\n")



pass_df = out_df.loc[out_df['검수결과'] == "통과"]
pass_df = pass_df.loc[pass_df['scene'] == "동적"]
pass_df

,clip 이름,가공자,검수일자,검수결과,반려사유,id 개수,객체 개수,박스 개수,오류 id 개수,오류 박스 총계,오류 id 목록,scene
5,extract_2022-08-02-18-18-53_Clip_00003,박세령,2022-08-24,통과,통과,21,406,406,0,0,없음,동적
15,extract_2022-08-02-18-18-53_Clip_00046,박민석,2022-08-24,통과,통과,8,203,203,0,0,없음,동적
20,extract_2022-08-02-18-18-53_Clip_00062,조재윤,2022-08-24,통과,통과,21,1201,1201,0,0,없음,동적
37,extract_2022-08-02-18-18-53_Clip_00059,박민석,2022-08-24,통과,통과,14,757,757,0,0,없음,동적
51,extract_2022-08-03-14-26-09_Clip_00000,조재윤,2022-08-24,통과,통과,1,100,100,0,0,없음,동적
...,...,...,...,...,...,...,...,...,...,...,...,...
182,extract_2022-08-03-15-44-05_Clip_00137,박세령,2022-08-24,통과,통과,3,300,300,0,0,없음,동적
183,extract_2022-08-03-14-48-26_Clip_00043,강려명,2022-08-24,통과,통과,1,100,100,0,0,없음,동적
184,extract_2022-08-03-15-44-05_Clip_00120,이정모,2022-08-24,통과,통과,2,200,200,0,0,없음,동적
185,extract_2022-08-03-15-44-05_Clip_00140,강려명,2022-08-24,통과,통과,3,300,300,0,0,없음,동적


In [147]:
super_base = r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill"
res = glob.glob(f'{super_base}/**/*.zip', recursive=True)
passed = list(pass_df["clip 이름"])
dst = r"C:\aivill_sey_ryeong\labeling\라벨링툴\검수결과\Aivill\통과클립"

for r in res:
    
    filename = (r.split("\\"))[-1]
    filename = filename[:-4]
    print(filename)
    if filename in passed:
        shutil.copyfile(r,f"{dst}/{filename}.zip")
    

extract_2022-08-02-18-18-53_Clip_00004
extract_2022-08-02-18-18-53_Clip_00011
extract_2022-08-02-18-18-53_Clip_00035
extract_2022-08-02-18-18-53_Clip_00056
extract_2022-08-02-18-18-53_Clip_00001
extract_2022-08-02-18-18-53_Clip_00003
extract_2022-08-02-18-18-53_Clip_00010
extract_2022-08-02-18-18-53_Clip_00012
extract_2022-08-02-18-18-53_Clip_00014
extract_2022-08-02-18-18-53_Clip_00017
extract_2022-08-02-18-18-53_Clip_00019
extract_2022-08-02-18-18-53_Clip_00021
extract_2022-08-02-18-18-53_Clip_00027
extract_2022-08-02-18-18-53_Clip_00028
extract_2022-08-02-18-18-53_Clip_00041
extract_2022-08-02-18-18-53_Clip_00046
extract_2022-08-02-18-18-53_Clip_00047
extract_2022-08-02-18-18-53_Clip_00053
extract_2022-08-02-18-18-53_Clip_00060
extract_2022-08-02-18-18-53_Clip_00061
extract_2022-08-02-18-18-53_Clip_00062
extract_2022-08-02-18-18-53_Clip_00076
extract_2022-08-01-16-09-39_Clip_00008
extract_2022-08-01-16-09-39_Clip_00013
extract_2022-08-01-16-09-39_Clip_00016
extract_2022-08-01-16-09-